## A、准备阶段
数据获取->数据探索->清洗数据->特征选择

### A1、数据获取

* 爬虫或其他手段获取
* 使用 head 查看前几行数据（默认是前 5 行）；
* 使用 tail 查看后几行数据（默认是最后 5 行）。

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
train_data = pd.read_csv('./titanic/train.csv', index_col = 0)
test_data = pd.read_csv('./titanic/test.csv', index_col = 0)

In [ ]:
train_data.head(3)

In [ ]:
test_data.head(3)

### A2、数据探索

* 使用 info() 了解数据表的基本情况：行数、列数、每列的数据类型、数据完整度；
* 使用 describe() 了解数据表的统计情况：总数、平均值、标准差、最小值、最大值等；
* 使用 describe(include=[‘O’]) 查看字符串类型（非数字）的整体情况。

In [ ]:
print(train_data.info())
print('-'*50)
print(train_data.describe())
print('-'*50)
print(train_data.describe(include=['O']))

In [ ]:
print(test_data.info())
print('-'*50)
print(test_data.describe())
print('-'*50)
print(test_data.describe(include=['O']))

### A3、清洗数据

In [ ]:
# train_data,test_tada中age段缺失数据，用平均数填补（具体填充方法按实际情况需求）
train_data['Age'].fillna(train_data['Age'].mean(), inplace = True)
test_data['Age'].fillna(test_data['Age'].mean(), inplace = True)

In [ ]:
# test_data中Fare(船票价格)段缺失数据，用平均数填补（具体填充方法按实际情况需求）
test_data['Fare'].fillna(test_data['Fare'].mean(), inplace = True)

In [ ]:
# Cabin为船舱，有大量的缺失值,暂时无法补齐；
# Embarked为登陆港口，有少量的缺失值
train_data['Embarked'].value_counts()

In [ ]:
train_data['Embarked'].fillna('S', inplace = True)

In [ ]:
train_data.info()

In [ ]:
test_data.info()

In [ ]:
# Cabin为船舱，有大量的缺失值
train_data['Cabin'].value_counts()

### A4、特征选择
选择影响预测的特征（字段），构建分类器：

PassengerId, Name, Ticket, Cabin等字段对分类器没什么作用，可以舍弃；
剩下的字段Pclass, Sex, Age, SibSp, Parch, Fare, Embarked放到分类器中

In [ ]:
# 特征选择
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
train_features = train_data[features]
test_features = test_data[features]

# 测试器标签
train_labels = train_data['Survived']

train_features.head(6)

In [ ]:
# 将特征包含的字符串转化成数值类型，以便后续运算，类型少可以使用独热编码
from sklearn.feature_extraction import DictVectorizer

vec = DictVectorizer(sparse = False)
train_features = vec.fit_transform(train_features.to_dict(orient = 'records'))
test_features=vec.transform(test_features.to_dict(orient='records'))

In [ ]:
# 特征矩阵
train_features

In [ ]:
# 查看属性
vec.feature_names_

## B、分类建模阶段
数学模型->模型评估->数据可视化->报告

### B1、建立模型

sklearn建模三补：
* 实例化：建立评估模型对象（参数）
* 训练：通过模型接口训练模型（数据属性、数据接口）
* 返回：通过接口返回需要的信息

In [ ]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(criterion = 'entropy', random_state = 30, splitter = 'random')
clf = clf.fit(train_features, train_labels)

In [ ]:
# 决策树预测
pred_labels = clf.predict(test_features)

In [ ]:
# 得到决策树准确率
acc_decision_tree = round(clf.score(train_features, train_labels), 6)
print(u'score准确率为 %.4lf' % acc_decision_tree)

In [ ]:
import numpy as np
from sklearn.model_selection import cross_val_score
# 使用K折交叉验证 统计决策树准确率
print(u'cross_val_score准确率为 %.4lf' % np.mean(cross_val_score(clf, train_features, train_labels, cv=10)))

In [ ]:
class_name = ['生', '死']

from sklearn import tree
import graphviz

date = tree.export_graphviz(clf
                           ,feature_names = vec.feature_names_
                           ,class_names = class_name
                           ,filled = True
                           ,rounded = True
                           ,out_file = None)
graph = graphviz.Source(date)

In [ ]:
graph

In [ ]:
clf.feature_importances_  # 打印最重要属性

In [ ]:
[*zip(vec.feature_names_,clf.feature_importances_)]

### B2、模型评估

### B3、数据可视化

### B4、报告